In [57]:
# Step 1:  Load Data
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
file_path= r"C:\Applied_GenAI\Course-4-AGS Advanced Generative AI Building LLM Applications\Assessment\AI_Powered_HR_Assistance\nestle_hr_policy.pdf"
loader = PyPDFLoader(file_path)
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

In [58]:
# Step 2: Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)

In [60]:
#Step 3: Load OpenAI API Key
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [61]:
# %% Step 4: Embed and vectorize and store
import os
from openai import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
persist_directory = r"C:\Applied_GenAI\Jupyter_Notebooks\basic_langchain\chroma_storage"
tiktoken_cache_dir = r"C:\Applied_GenAI\Jupyter_Notebooks\temp"
os.environ["TIKTOKEN_CACHE_DIR"] = tiktoken_cache_dir
embedding = OpenAIEmbeddings(api_key=api_key)
vectordb = Chroma.from_texts(
    texts=splits,
    embedding=embedding,
    persist_directory=persist_directory
)
vectordb.persist()

error uploading: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1020)')))


In [62]:
# Retrieve the answer
from langchain.chat_models import ChatOpenAI
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0), 
    chain_type="stuff", 
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
)

In [65]:
# Method to ask the question
def chat_with_chatgpt(usermessage):
    result=qa(usermessage)
    return result["result"]

In [66]:
# Integration with gradio
import gradio as gr

demo = gr.Interface(fn=chat_with_chatgpt,
                    inputs=gr.Textbox(label= "What can I help with?"),
                    outputs=gr.Textbox(label= "AI-generated response"),
                    title="Nestle AI-Powered HR Assistant")
demo.launch()


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Using existing dataset file at: .gradio\flagged\dataset1.csv
